Experiment 9 600 images of compressed dataset

Experiment 8 is not working on compressed dataset of 1000 images each feature
Experiment 7 is the working on all 9 smaller datasets..
Experiment 6 is working fine with 8 smaller dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import the necessary packages
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import tempfile

class LearningRateFinder:
	def __init__(self, model, stopFactor=4, beta=0.98):
		# store the model, stop factor, and beta value (for computing
		# a smoothed, average loss)
		self.model = model
		self.stopFactor = stopFactor
		self.beta = beta

		# initialize our list of learning rates and losses,
		# respectively
		self.lrs = []
		self.losses = []

		# initialize our learning rate multiplier, average loss, best
		# loss found thus far, current batch number, and weights file
		self.lrMult = 1
		self.avgLoss = 0
		self.bestLoss = 1e9
		self.batchNum = 0
		self.weightsFile = None

	def reset(self):
		# re-initialize all variables from our constructor
		self.lrs = []
		self.losses = []
		self.lrMult = 1
		self.avgLoss = 0
		self.bestLoss = 1e9
		self.batchNum = 0
		self.weightsFile = None

	def is_data_iter(self, data):
		# define the set of class types we will check for
		iterClasses = ["NumpyArrayIterator", "DirectoryIterator",
			 "DataFrameIterator", "Iterator", "Sequence"]

		# return whether our data is an iterator
		return data.__class__.__name__ in iterClasses

	def on_batch_end(self, batch, logs):
		# grab the current learning rate and add log it to the list of
		# learning rates that we've tried
		lr = K.get_value(self.model.optimizer.lr)
		self.lrs.append(lr)

		# grab the loss at the end of this batch, increment the total
		# number of batches processed, compute the average average
		# loss, smooth it, and update the losses list with the
		# smoothed value
		l = logs["loss"]
		self.batchNum += 1
		self.avgLoss = (self.beta * self.avgLoss) + ((1 - self.beta) * l)
		smooth = self.avgLoss / (1 - (self.beta ** self.batchNum))
		self.losses.append(smooth)

		# compute the maximum loss stopping factor value
		stopLoss = self.stopFactor * self.bestLoss

		# check to see whether the loss has grown too large
		if self.batchNum > 1 and smooth > stopLoss:
			# stop returning and return from the method
			self.model.stop_training = True
			return

		# check to see if the best loss should be updated
		if self.batchNum == 1 or smooth < self.bestLoss:
			self.bestLoss = smooth

		# increase the learning rate
		lr *= self.lrMult
		K.set_value(self.model.optimizer.lr, lr)

	def find(self, trainData, startLR, endLR, epochs=None,
		stepsPerEpoch=None, batchSize=32, sampleSize=2048,
		verbose=1):
		# reset our class-specific variables
		self.reset()

		# determine if we are using a data generator or not
		useGen = self.is_data_iter(trainData)

		# if we're using a generator and the steps per epoch is not
		# supplied, raise an error
		if useGen and stepsPerEpoch is None:
			msg = "Using generator without supplying stepsPerEpoch"
			raise Exception(msg)

		# if we're not using a generator then our entire dataset must
		# already be in memory
		elif not useGen:
			# grab the number of samples in the training data and
			# then derive the number of steps per epoch
			numSamples = len(trainData[0])
			stepsPerEpoch = np.ceil(numSamples / float(batchSize))

		# if no number of training epochs are supplied, compute the
		# training epochs based on a default sample size
		if epochs is None:
			epochs = int(np.ceil(sampleSize / float(stepsPerEpoch)))

		# compute the total number of batch updates that will take
		# place while we are attempting to find a good starting
		# learning rate
		numBatchUpdates = epochs * stepsPerEpoch

		# derive the learning rate multiplier based on the ending
		# learning rate, starting learning rate, and total number of
		# batch updates
		self.lrMult = (endLR / startLR) ** (1.0 / numBatchUpdates)

		# create a temporary file path for the model weights and
		# then save the weights (so we can reset the weights when we
		# are done)
		self.weightsFile = tempfile.mkstemp()[1]
		self.model.save_weights(self.weightsFile)

		# grab the *original* learning rate (so we can reset it
		# later), and then set the *starting* learning rate
		origLR = K.get_value(self.model.optimizer.lr)
		K.set_value(self.model.optimizer.lr, startLR)

		# construct a callback that will be called at the end of each
		# batch, enabling us to increase our learning rate as training
		# progresses
		callback = LambdaCallback(on_batch_end=lambda batch, logs:
			self.on_batch_end(batch, logs))

		# check to see if we are using a data iterator
		if useGen:
			self.model.fit_generator(
				trainData,
				steps_per_epoch=stepsPerEpoch,
				epochs=epochs,
				verbose=verbose,
				callbacks=[callback])

		# otherwise, our entire training data is already in memory
		else:
			# train our model using Keras' fit method
			self.model.fit(
				trainData[0], trainData[1],
				batch_size=batchSize,
				epochs=epochs,
				callbacks=[callback],
				verbose=verbose)

		# restore the original model weights and learning rate
		self.model.load_weights(self.weightsFile)
		K.set_value(self.model.optimizer.lr, origLR)

	def plot_loss(self, skipBegin=10, skipEnd=1, title=""):
		# grab the learning rate and losses values to plot
		lrs = self.lrs[skipBegin:-skipEnd]
		losses = self.losses[skipBegin:-skipEnd]

		# plot the learning rate vs. loss
		plt.plot(lrs, losses)
		plt.xscale("log")
		plt.xlabel("Learning Rate (Log Scale)")
		plt.ylabel("Loss")

		# if the title is not empty, add it to the plot
		if title != "":
			plt.title(title)

In [ ]:
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
import numpy as np


class CyclicLR(Callback):
	"""This callback implements a cyclical learning rate policy (CLR).
	The method cycles the learning rate between two boundaries with
	some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
	The amplitude of the cycle can be scaled on a per-iteration or
	per-cycle basis.
	This class has three built-in policies, as put forth in the paper.
	"triangular":
		A basic triangular cycle w/ no amplitude scaling.
	"triangular2":
		A basic triangular cycle that scales initial amplitude by half each cycle.
	"exp_range":
		A cycle that scales initial amplitude by gamma**(cycle iterations) at each
		cycle iteration.
	For more detail, please see paper.

	# Example
		```python
			clr = CyclicLR(base_lr=0.001, max_lr=0.006,
								step_size=2000., mode='triangular')
			model.fit(X_train, Y_train, callbacks=[clr])
		```

	Class also supports custom scaling functions:
		```python
			clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
			clr = CyclicLR(base_lr=0.001, max_lr=0.006,
								step_size=2000., scale_fn=clr_fn,
								scale_mode='cycle')
			model.fit(X_train, Y_train, callbacks=[clr])
		```
	# Arguments
		base_lr: initial learning rate which is the
			lower boundary in the cycle.
		max_lr: upper boundary in the cycle. Functionally,
			it defines the cycle amplitude (max_lr - base_lr).
			The lr at any cycle is the sum of base_lr
			and some scaling of the amplitude; therefore
			max_lr may not actually be reached depending on
			scaling function.
		step_size: number of training iterations per
			half cycle. Authors suggest setting step_size
			2-8 x training iterations in epoch.
		mode: one of {triangular, triangular2, exp_range}.
			Default 'triangular'.
			Values correspond to policies detailed above.
			If scale_fn is not None, this argument is ignored.
		gamma: constant in 'exp_range' scaling function:
			gamma**(cycle iterations)
		scale_fn: Custom scaling policy defined by a single
			argument lambda function, where
			0 <= scale_fn(x) <= 1 for all x >= 0.
			mode paramater is ignored
		scale_mode: {'cycle', 'iterations'}.
			Defines whether scale_fn is evaluated on
			cycle number or cycle iterations (training
			iterations since start of cycle). Default is 'cycle'.
	"""

	def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
				 gamma=1., scale_fn=None, scale_mode='cycle'):
		super(CyclicLR, self).__init__()

		self.base_lr = base_lr
		self.max_lr = max_lr
		self.step_size = step_size
		self.mode = mode
		self.gamma = gamma
		if scale_fn == None:
			if self.mode == 'triangular':
				self.scale_fn = lambda x: 1.
				self.scale_mode = 'cycle'
			elif self.mode == 'triangular2':
				self.scale_fn = lambda x: 1 / (2. ** (x - 1))
				self.scale_mode = 'cycle'
			elif self.mode == 'exp_range':
				self.scale_fn = lambda x: gamma ** (x)
				self.scale_mode = 'iterations'
		else:
			self.scale_fn = scale_fn
			self.scale_mode = scale_mode
		self.clr_iterations = 0.
		self.trn_iterations = 0.
		self.history = {}

		self._reset()

	def _reset(self, new_base_lr=None, new_max_lr=None,
			   new_step_size=None):
		"""Resets cycle iterations.
		Optional boundary/step size adjustment.
		"""
		if new_base_lr != None:
			self.base_lr = new_base_lr
		if new_max_lr != None:
			self.max_lr = new_max_lr
		if new_step_size != None:
			self.step_size = new_step_size
		self.clr_iterations = 0.

	def clr(self):
		cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
		x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
		if self.scale_mode == 'cycle':
			return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
		else:
			return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
				self.clr_iterations)

	def on_train_begin(self, logs={}):
		logs = logs or {}

		if self.clr_iterations == 0:
			K.set_value(self.model.optimizer.lr, self.base_lr)
		else:
			K.set_value(self.model.optimizer.lr, self.clr())

	def on_batch_end(self, epoch, logs=None):

		logs = logs or {}
		self.trn_iterations += 1
		self.clr_iterations += 1

		self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
		self.history.setdefault('iterations', []).append(self.trn_iterations)

		for k, v in logs.items():
			self.history.setdefault(k, []).append(v)

		K.set_value(self.model.optimizer.lr, self.clr())


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/classification9/600ImagesComporessedDataSet.zip" -d "/content/drive/MyDrive/Colab Notebooks/classification9/Datasets"
#1 min for datasets

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet/Alcohol/590.jpg  
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet/Alcohol/591.jpg  
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet/Alcohol/592.jpg  
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet/Alcohol/593.jpg  
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet/Alcohol/594.jpg  
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet/Alcohol/595.jpg  
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet/Alcohol/596.jpg  
  inflating: /content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesCo

In [ ]:
#!rm -rf '/content/drive/MyDrive/Colab Notebooks/classification4/Datasets/Flood'

In [ ]:
# import the necessary packages
import os

# initialize the path to the input directory containing our dataset
# of images
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/classification9/Datasets/600ImagesComporessedDataSet"

# initialize the class labels in the dataset
CLASSES = ["Alcohol",  "Drunk", "Earthquake", "Fire", "Flood", "Harrasment" , "Robbery", "Smoking" , "Violence"]

# 
# define the size of the training, validation (which comes from the
# train split), and testing splits, respectively
TRAIN_SPLIT = 0.9
VAL_SPLIT = 0.1
TEST_SPLIT = 0.1

# define the minimum learning rate, maximum learning rate, batch size,
# step size, CLR method, and number of epochs
MIN_LR = 1e-6
MAX_LR = 1e-4
BATCH_SIZE = 16
STEP_SIZE = 8
CLR_METHOD = "triangular"
NUM_EPOCHS = 120

# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["/content/drive/MyDrive/Colab Notebooks/classification9/output", "Secure_Campus.model"])

# define the path to the output learning rate finder plot, training
# history plot and cyclical learning rate plot
LRFIND_PLOT_PATH = os.path.sep.join(["content/drive/MyDrive/Colab Notebooks/classificaiton9/output", "lrfind_plot.png"])
TRAINING_PLOT_PATH = os.path.sep.join(["content/drive/MyDrive/Colab Notebooks/classificaiton9/output", "training_plot.png"])
CLR_PLOT_PATH = os.path.sep.join(["content/drive/MyDrive/Colab Notebooks/classificaiton9/output", "clr_plot.png"])

In [ ]:
# USAGE
# python train.py --lr-find 1
# python train.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#from learningratefinder import LearningRateFinder
#from clr_callback import CyclicLR
#from pyimagesearch import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import sys
import os
import easydict

In [ ]:
# construct the argument parser and parse the arguments
#parser = argparse.ArgumentParser()
#parser.add_argument("-f", "--lr_find", type=int, default=0, help="whether or not to find optimal learning rate")
#args = vars(parser.parse_args())

args = easydict.EasyDict({
    "lr_find": 0
})

# grab the paths to all images in our dataset directory and initialize
# our lists of images and class labels
print("[INFO] loading images...")
imagePaths = list(paths.list_images(DATASET_PATH))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label
	label = imagePath.split(os.path.sep)[-2]

	#labelprint = "======================================  "+label+"  ======================================"
	#print(labelprint)

	# load the image, convert it to RGB channel ordering, and resize
	# it to be a fixed 224x224 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
 
	#print(imagePath)
 
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)
  
	#2min of time for datasets
	
  

[INFO] loading images...


In [ ]:
def Hundred_image_processing(data_rec):

    imgsl = []
    i = 0

    for oneimage in data_rec:
        imgsl.append(oneimage)
        i += 1
        if i == 19:
            imgs = np.array(imgsl, dtype="float32")
            imgsl = []
        if i % 100 == 0:
            i = 20
            imgst = np.array(imgsl, dtype="float32")
            imgsl = []
            imgs = np.concatenate((imgs, imgst), axis=0)
            print('Hundred images processed!')
            print('len: imgls: ', len(imgsl))
            print('imgs shape: ', imgs.shape)

    imgst = np.array(imgsl, dtype="float32")
    imgsl = []
    imgs = np.append(imgs, imgst, axis=0)

    return imgs

In [ ]:
# convert the data and labels to NumPy arrays
print("[INFO] processing data...")
data = np.array(data, dtype="float32")
#data = Hundred_image_processing(data)
labels = np.array(labels)

[INFO] processing data...


In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=TEST_SPLIT, random_state=42)

# take the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY,
	test_size=VAL_SPLIT, random_state=84)





In [ ]:
# initialize the training data augmentation object
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=MIN_LR, momentum=0.9)

#model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,	metrics=["accuracy"])
model.compile(loss="categorical_crossentropy", optimizer=opt,	metrics=["accuracy"])

# check to see if we are attempting to find an optimal learning rate
# before training for the full number of epochs
if args["lr_find"] > 0:
	# initialize the learning rate finder and then train with learning
	# rates ranging from 1e-10 to 1e+1
	print("[INFO] finding learning rate...")
	lrf = LearningRateFinder(model)
	lrf.find(
		aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
		1e-10, 1e+1,
		stepsPerEpoch=np.ceil((trainX.shape[0] / float(BATCH_SIZE))),
		epochs=20,
		batchSize=BATCH_SIZE)
 
	# plot the loss for the various learning rates and save the
	# resulting plot to disk
	lrf.plot_loss()
	plt.savefig(LRFIND_PLOT_PATH)
 
	# gracefully exit the script so we can adjust our learning rates
	# in the and then train the network for our full set of
	# epochs
	print("[INFO] learning rate finder complete")
	print("[INFO] examine plot and adjust learning rates before training")
	sys.exit(0)

# otherwise, we have already defined a learning rate space to train
# over, so compute the step size and initialize the cyclic learning
# rate method
stepSize = STEP_SIZE * (trainX.shape[0] // BATCH_SIZE)
clr = CyclicLR(
	mode=CLR_METHOD,
	base_lr=MIN_LR,
	max_lr=MAX_LR,
	step_size=stepSize)



[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
	validation_data=(valX, valY),
	steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
	epochs=NUM_EPOCHS,
	callbacks=[clr],
	verbose=1)

[INFO] training network...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/120
273/273 [==============================] - 68s 214ms/step - loss: 13.5674 - accuracy: 0.2731 - val_loss: 3.1098 - val_accuracy: 0.6543
Epoch 2/120
273/273 [==============================] - 52s 190ms/step - loss: 4.3697 - accuracy: 0.5207 - val_loss: 1.2514 - val_accuracy: 0.7675
Epoch 3/120
273/273 [==============================] - 51s 188ms/step - loss: 2.1410 - accuracy: 0.5886 - val_loss: 0.8050 - val_accuracy: 0.8251
Epoch 4/120
273/273 [==============================] - 51s 186ms/step - loss: 1.4719 - accuracy: 0.6303 - val_loss: 0.6242 - val_accuracy: 0.8436
Epoch 5/120
273/273 [==============================] - 50s 184ms/step - loss: 1.2138 - accuracy: 0.6804 - val_loss: 0.5590 - val_accuracy: 0.8683
Epoch 6/120
273/273 [==============================] - 51s 185ms/step - loss: 1.0364 - accuracy: 0.7079 - val_loss: 0.5517 - val_accuracy: 0.8539
Epoch 7/120
273/273 [==============================] - 50s 184ms/step - loss: 0.9510 - accuracy: 0.7263 - val_loss: 0.4775 

In [ ]:
# evaluate the network and show a classification report
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BATCH_SIZE)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=CLASSES))

# serialize the model to disk
print("[INFO] serializing network to '{}'...".format(MODEL_PATH))
model.save(MODEL_PATH)



[INFO] evaluating network...
34/34 [==============================] - 4s 121ms/step
              precision    recall  f1-score   support

     Alcohol       0.97      0.91      0.94        74
       Drunk       0.98      1.00      0.99        56
  Earthquake       0.98      0.94      0.96        64
        Fire       1.00      0.98      0.99        58
       Flood       0.93      0.98      0.95        51
  Harrasment       0.97      1.00      0.98        65
     Robbery       0.96      0.96      0.96        57
     Smoking       0.94      0.98      0.96        62
    Violence       0.98      0.98      0.98        53

    accuracy                           0.97       540
   macro avg       0.97      0.97      0.97       540
weighted avg       0.97      0.97      0.97       540

[INFO] serializing network to '/content/drive/MyDrive/Colab Notebooks/classification9/output/Secure_Campus.model'...


In [ ]:
from google.colab import files


# construct a plot that plots and saves the training history
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

plt.savefig("training_plot.png")
#plt.savefig(TRAINING_PLOT_PATH)
files.download("training_plot.png") 

# plot the learning rate history
N = np.arange(0, len(clr.history["lr"]))
plt.figure()
plt.plot(N, clr.history["lr"])
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Training Iterations")
plt.ylabel("Learning Rate")

plt.savefig("clr_plot.png")
#plt.savefig(CLR_PLOT_PATH)
files.download("clr_plot.png") 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import load_model
#from pyimagesearch import config
from collections import deque
import numpy as np
import argparse
import cv2
import easydict
# import the necessary packages
import os

In [ ]:
# import the necessary packages
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import easydict
from google.colab.patches import cv2_imshow

In [ ]:
args = easydict.EasyDict({
    #"dataset": "/content/drive/MyDrive/Colab Notebooks/classificaiton2/Cyclone_Wildfire_Flood_Earthquake_Database",
    "model": "/content/drive/MyDrive/Colab Notebooks/classification9/output/Secure_Campus.model",
    "label-bin": "/content/drive/MyDrive/Colab Notebooks/classification9/model/lb.pickle",
    #"input": "/content/drive/MyDrive/Colab Notebooks/classification4/Demo video creation 2.mp4",
	   # "input": "/content/drive/MyDrive/Colab Notebooks/classification4/Demo video creation.mp4",
	  #"output": "/content/drive/MyDrive/Colab Notebooks/classification4/output/Demo_video_creation_2.avi",
  	# "output": "/content/drive/MyDrive/Colab Notebooks/classification4/output/OUTPUT_Demo_video_creation.avi",

    "input": "/content/drive/MyDrive/Colab Notebooks/classification9/DEMOVIDEO.mp4",
	  "output": "/content/drive/MyDrive/Colab Notebooks/classification9/output/DEMOVIDEO exp 9 OUTPUT.avi",
    "size": 128
})

In [ ]:


# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(args["model"])
# lb = pickle.loads(open(args["label_bin"], "rb").read())
lb = ["Alcohol",  "Drunk", "Earthquake", "Fire", "Flood", "Harrasment" , "Robbery","Smoking" , "Violence"]

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=args["size"])

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(args["input"])
writer = None
(W, H) = (None, None)
#client = Client("ACea4cecca40ebb1bf4594098d5cef4541", "32789639585561088d5937514694e115") #update from twilio
prelabel = ''
ok = 'Normal'
fi_label = []
framecount = 0


# loop over frames from the video file stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break

	# if the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]
	framecount = framecount+1
	# clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction
	output = frame.copy()
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	frame = cv2.resize(frame, (224, 224)).astype("float32")
	frame -= mean

	# make predictions on the frame and then update the predictions
	# queue
	preds = model.predict(np.expand_dims(frame, axis=0))[0]
#	print('Preds = :', preds)
	
#	total = (preds[0]+ preds[1]+preds[2] + preds[3]+ preds[4]+preds[5])
#	maximum = max(preds)
#	rest = total - maximum
    
#	diff = (.8*maximum) - (.1*rest)
#	print('Difference of prob ', diff)
#	th = 100
#	if diff > .60:
#		th = diff
#	print('Old threshold = ', th)
    
    
	prediction = preds.argmax(axis=0)
	Q.append(preds)

	# perform prediction averaging over the current history of
	# previous predictions
	results = np.array(Q).mean(axis=0)
	print('Results = ', results)
	maxprob = np.max(results)
	print('Maximun Probability = ', maxprob)
	i = np.argmax(results)
	label = lb[i]

	rest = 1 - maxprob
    
	diff = (maxprob) - (rest)
	print('Difference of prob ', diff)
	th = 100
	if diff > .70:
		th = diff
        
	if (preds[prediction]) < th:
		text = "Alert : {} - {:.2f}%".format((ok), 100 - (maxprob * 100))
		cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)
	else:
		fi_label = np.append(fi_label, label)
		text = "Alert : {} - {:.2f}%".format((label), maxprob * 100)
		cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5) 
#		if label != prelabel:
#			client.messages.create(to="<+country code>< receiver mobile number>", #for example +918255555555
#                       from_="+180840084XX", #sender number can be coped from twilo
#                       body='\n'+ str(text) +'\n Satellite: ' + str(camid) + '\n Orbit: ' + location)
		prelabel = label    


	# check if the video writer is None
	if writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 30,
			(W, H), True)

	# write the output frame to disk
	writer.write(output)

	# show the output image
	# cv2_imshow( "Output")
	#cv2.imshow("Output", output)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

print('Frame count', framecount)
print('Count label', fi_label)

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

Streaming output truncated to the last 5000 lines.
Difference of prob  0.32238292694091797
1/1 [==============================] - 0s 25ms/step
Results =  [1.22037875e-02 2.32298589e-06 4.38954541e-03 6.41260194e-05
 1.08301196e-04 3.79560627e-02 2.82075346e-01 1.62018870e-04
 6.63038433e-01]
Maximun Probability =  0.66303843
Difference of prob  0.326076865196228
1/1 [==============================] - 0s 22ms/step
Results =  [1.2157724e-02 2.3229986e-06 4.3895449e-03 6.4126019e-05 1.0830088e-04
 3.7922118e-02 2.8089377e-01 1.6199509e-04 6.6430002e-01]
Maximun Probability =  0.6643
Difference of prob  0.32860004901885986
1/1 [==============================] - 0s 23ms/step
Results =  [1.2130181e-02 2.3230007e-06 4.3895449e-03 6.4126019e-05 1.0830085e-04
 3.7825026e-02 2.7884117e-01 1.6198520e-04 6.6647726e-01]
Maximun Probability =  0.66647726
Difference of prob  0.3329545259475708
1/1 [==============================] - 0s 22ms/step
Results =  [1.20798796e-02 2.32300999e-06 4.38954495e-03